In [ ]:
#########
# The purpose of this script is to do TFCE analysis on functional connectivity data and compare the results to results 
# from Conn
# Tom Possidente - January 2025
#########

In [3]:
import numpy as np
import pandas as pd
import scipy
from mne.stats import permutation_cluster_1samp_test

**Load in gPPI betas for each condition (for each subj for each ROI for each connection) from mat file produced by Conn**

In [18]:
ROI_data_fpath = '/projectnb/somerslab/tom/projects/spacetime_network/data/conn_toolbox_folder/conn_localizer_task/results/secondlevel/avsm_gPPI/AllSubjects/aA(1).vA(-1)/conn_gPPI_betadiffs.mat'
mat = scipy.io.loadmat(ROI_data_fpath)

beta_diffs = mat['beta_diffs']
ROI_names = np.squeeze(mat['ROI_names'])
#adjmat = mat['adj_mat']
adjmat = np.zeros([32,32])

# convert numpy adjmat to scipy sparse matrix
adjmat_sp = scipy.sparse.coo_matrix(adjmat)

print(np.shape(beta_diffs)) # subj x ROI x ROI_connection

print(ROI_names)

(21, 32, 32)
[array(['avsm_ROIs.CO (L)'], dtype='<U16')
 array(['avsm_ROIs.FO (L)'], dtype='<U16')
 array(['avsm_ROIs.cIFSG (L)'], dtype='<U19')
 array(['avsm_ROIs.cmSFG (L)'], dtype='<U19')
 array(['avsm_ROIs.iPCS (L)'], dtype='<U18')
 array(['avsm_ROIs.midIFS (L)'], dtype='<U20')
 array(['avsm_ROIs.pAud (L)'], dtype='<U18')
 array(['avsm_ROIs.pVis (L)'], dtype='<U18')
 array(['avsm_ROIs.sPCS (L)'], dtype='<U18')
 array(['avsm_ROIs.tgPCS (L)'], dtype='<U19')
 array(['avsm_ROIs.sm_sPCS (L)'], dtype='<U21')
 array(['avsm_ROIs.sm_iPCS (L)'], dtype='<U21')
 array(['avsm_ROIs.sm_midFSG (L)'], dtype='<U23')
 array(['avsm_ROIs.sm_aINS (L)'], dtype='<U21')
 array(['avsm_ROIs.sm_preSMA (L)'], dtype='<U23')
 array(['avsm_ROIs.sm_dACC (L)'], dtype='<U21')
 array(['avsm_ROIs.CO (R)'], dtype='<U16')
 array(['avsm_ROIs.FO (R)'], dtype='<U16')
 array(['avsm_ROIs.cIFSG (R)'], dtype='<U19')
 array(['avsm_ROIs.cmSFG (R)'], dtype='<U19')
 array(['avsm_ROIs.iPCS (R)'], dtype='<U18')
 array(['avsm_ROIs.mi

**Do TFCE**

In [24]:
tfce_thresholds = dict(start=0, step=0.01)
n_permutations = 10000
t_tfce, clusters, p_tfce, H0 = permutation_cluster_1samp_test(
    beta_diffs,
    n_jobs=None,
    t_power=2,
    threshold=None,
    adjacency=adjmat_sp,
    n_permutations=n_permutations,
    out_type="mask",
)

Using a threshold of 2.085963
stat_fun(H1): min=nan max=nan
Running initial clustering …
Found 132 clusters


/scratch/3185826.1.onrcc-m256/ipykernel_3291625/1110698551.py:3: RuntimeWarning: Provided stat_fun does not treat variables independently. Setting buffer_size to None.
  t_tfce, clusters, p_tfce, H0 = permutation_cluster_1samp_test(


  0%|          | Permuting : 0/9999 [00:00<?,       ?it/s]

In [27]:
sig_clusters = np.array(clusters)[p_tfce<0.05]

In [28]:
for cc in range(sig_clusters.shape[0]):
    cluster = sig_clusters[cc]
    inds = np.where(cluster)
    print('cluster ' + str(cc))
    for ii in range(len(inds[0])):
        print(ROI_names[inds[0][ii]][0] + ' <-> ' + ROI_names[inds[1][ii]][0] + ' ' + str(np.round(t_tfce[inds[0][ii], inds[1][ii]],2)))
    print('')
    


cluster 0
avsm_ROIs.sm_aINS (L) <-> avsm_ROIs.cIFSG (L) -4.57
avsm_ROIs.sm_preSMA (L) <-> avsm_ROIs.cIFSG (L) -5.49

cluster 1
avsm_ROIs.cmSFG (R) <-> avsm_ROIs.sm_iPCS (R) -3.71
avsm_ROIs.iPCS (R) <-> avsm_ROIs.sm_iPCS (R) -3.91
avsm_ROIs.midIFS (R) <-> avsm_ROIs.sm_iPCS (R) -2.24

cluster 2
avsm_ROIs.midIFS (R) <-> avsm_ROIs.cmSFG (R) -5.76



In [23]:
p_tfce[p_tfce<0.05]

array([0.003, 0.037, 0.042])